In [154]:
import networkx as nx

G = nx.DiGraph()
G.add_nodes_from([
    ('src', {'size':0}),
    ('A', {'size': 5}),
    ('B', {'size': 10}),
    ('C', {'size': 7}),
    ('D', {'size': 8}),
    ('E', {'size': 3}),
    ('F', {'size': 20}),
    ('G', {'size': -4}),
    ('snk', {'size': 0}),
])

for i in "ABCDEFG":
    G.add_edge('src', i)
    G.add_edge(i, 'snk')
# Add directed edges
G.add_edges_from([
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'D'),
    ('A', 'E'),
    ('A', 'F'),
    ('A', 'G'),
    ('B', 'C'),
    ('B', 'D'),
    ('B', 'E'),
    ('B', 'F'),
    ('B', 'G'),
    ('C', 'D'),
    ('C', 'E'),
    ('C', 'F'),
    ('C', 'G'),
    ('D', 'E'),
    ('D', 'F'),
    ('D', 'G'),
    ('E', 'F'),
    ('E', 'G'),
    ('F', 'G'),
])

In [163]:


def min_diff(a, b):
    dp = [0] * (len(b) + 1)
    for i in range(1, len(b) + 1):
        dp[i] = float('inf')
        for j in range(i):
            dp[i] = min(dp[i], dp[j] + abs(a[j] - b[i - 1]))
    print(a, b)
    return dp[-1]

def path_sum(graph, path):
    return sum(graph.nodes[node]['size'] for node in path)

def closest_path(graph, weights, start, end):
    closest_path = None
    min_diff_val = float('inf')

    for path in nx.all_simple_paths(graph, source=start, target=end):
        path_weights = [graph.nodes[node]['size'] for node in path]
        diff = min_diff(path_weights, weights)
        if diff < min_diff_val:
            min_diff_val = diff
            closest_path = path

    return closest_path

if __name__ == "__main__":
    graph = nx.DiGraph()
    graph.add_nodes_from([(0, {'size': 1}), (1, {'size': 2}), (2, {'size': 3}),
                          (3, {'size': 4}), (4, {'size': 5})])
    graph.add_edges_from([(0, 1), (0, 2), (1, 3), (2, 3), (3, 4)])

    weights = [2, 3, 5]
    start = 0
    end = 4

    path = closest_path(graph, weights, start, end)
    if path:
        path_weights = [graph.nodes[node]['size'] for node in path]
        print("Closest path:", path)
        print("Path sizes:", path_weights)
    else:
        print("No path found")

[1, 2, 4, 5] [2, 3, 5]
[1, 3, 4, 5] [2, 3, 5]
Closest path: [0, 2, 3, 4]
Path sizes: [1, 3, 4, 5]


In [164]:
weights = [15,5]
path = closest_path(G, weights, 'src', 'snk')
if path:
    path_weights = [G.nodes[node]['size'] for node in path]
    print("Closest path:", path)
    print("Path weights:", path_weights)
else:
    print("No path found")

[0, 5, 0] [15, 5]
[0, 5, 10, 0] [15, 5]
[0, 5, 10, 7, 0] [15, 5]
[0, 5, 10, 7, 8, 0] [15, 5]
[0, 5, 10, 7, 8, 3, 0] [15, 5]
[0, 5, 10, 7, 8, 3, 20, 0] [15, 5]
[0, 5, 10, 7, 8, 3, 20, -4, 0] [15, 5]
[0, 5, 10, 7, 8, 3, -4, 0] [15, 5]
[0, 5, 10, 7, 8, 20, 0] [15, 5]
[0, 5, 10, 7, 8, 20, -4, 0] [15, 5]
[0, 5, 10, 7, 8, -4, 0] [15, 5]
[0, 5, 10, 7, 3, 0] [15, 5]
[0, 5, 10, 7, 3, 20, 0] [15, 5]
[0, 5, 10, 7, 3, 20, -4, 0] [15, 5]
[0, 5, 10, 7, 3, -4, 0] [15, 5]
[0, 5, 10, 7, 20, 0] [15, 5]
[0, 5, 10, 7, 20, -4, 0] [15, 5]
[0, 5, 10, 7, -4, 0] [15, 5]
[0, 5, 10, 8, 0] [15, 5]
[0, 5, 10, 8, 3, 0] [15, 5]
[0, 5, 10, 8, 3, 20, 0] [15, 5]
[0, 5, 10, 8, 3, 20, -4, 0] [15, 5]
[0, 5, 10, 8, 3, -4, 0] [15, 5]
[0, 5, 10, 8, 20, 0] [15, 5]
[0, 5, 10, 8, 20, -4, 0] [15, 5]
[0, 5, 10, 8, -4, 0] [15, 5]
[0, 5, 10, 3, 0] [15, 5]
[0, 5, 10, 3, 20, 0] [15, 5]
[0, 5, 10, 3, 20, -4, 0] [15, 5]
[0, 5, 10, 3, -4, 0] [15, 5]
[0, 5, 10, 20, 0] [15, 5]
[0, 5, 10, 20, -4, 0] [15, 5]
[0, 5, 10, -4, 0] [15, 5]
[0, 5,